In [13]:
from datetime import datetime
import pandas as pd

melbourne_file_path = 'melb_data.csv/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)
df = melbourne_data.describe()
# Самый старый и самый новый дом
print(df['YearBuilt']['min'], int(datetime.now().year - df['YearBuilt']['min']), (datetime.now().year - df['YearBuilt']['max']))



1196.0 829 7.0
